In [ ]:
# === Welcome Round ===============================
# Configure these and run the cell.

STUDENTS_CSV = "data/L_names.csv"    # path to your CSV
NAME_FIELD   = None                  # e.g. "name" — if None, auto-detect first non-empty header or "name"
QUESTIONS_TXT = "data/questions.txt" # path to .txt file with one question per line; or None if not used
EXTRA_RANDOM = 3                     # how many random questions to draw from .txt file
SEED         = None                  # set to an int for reproducible order (e.g., 42), or None for fresh shuffle
EXPORT_PATH  = None                  # e.g. "plan.csv" to save the randomized plan; or None to skip export

DEFAULT_QUESTIONS = [
    "Bei welcher Firma, Abteilung arbeiten Sie?",
    "Haben Sie schon Erfahrung im Programmieren, falls Ja, welche Sprache?",
]

# --- Implementation (no changes needed below) ---------------------------------
import csv
import random
from pathlib import Path

class _SafeDict(dict):
    def __missing__(self, key):
        return "{" + key + "}"

def _read_extra_questions(path):
    if not path:
        return []
    p = Path(path)
    if not p.exists():
        raise SystemExit(f"Questions file not found: {p}")
    with p.open("r", encoding="utf-8") as f:
        qs = [line.strip() for line in f if line.strip()]
    return qs

def _detect_name_field(header):
    lowered = [h.strip().lower() for h in header]
    if "name" in lowered:
        return header[lowered.index("name")]
    for h in header:
        if h.strip():
            return h
    return header[0] if header else None

def _read_students(csv_path, name_field):
    p = Path(csv_path)
    if not p.exists():
        raise SystemExit(f"Students CSV not found: {p}")
    with p.open("r", encoding="utf-8-sig", newline="") as f:
        reader = csv.DictReader(f)
        if reader.fieldnames is None:
            raise SystemExit("Students CSV has no header row.")
        name_col = name_field or _detect_name_field(reader.fieldnames)
        if not name_col:
            raise SystemExit("Could not determine a name column.")
        students = []
        seen_names = set()
        for row in reader:
            name = (row.get(name_col) or "").strip()
            if not name:
                continue
            if name in seen_names:
                continue
            seen_names.add(name)
            students.append(row)
    if not students:
        raise SystemExit("No students found in the CSV (check the name column).")
    return students, name_col

def _make_plan(students, name_col, base_questions, extra_questions, extra_random, rng):
    order = students[:]
    rng.shuffle(order)
    plan = []
    for row in order:
        # pick random extras
        chosen_extras = rng.sample(extra_questions, min(extra_random, len(extra_questions)))
        # keep DEFAULT_QUESTIONS in original order, extras after
        combined = base_questions + chosen_extras
        formatted = [q.format_map(_SafeDict(row)) for q in combined]
        plan.append((row[name_col].strip(), formatted))
    return plan

def _export_plan(plan, path):
    out = Path(path)
    out.parent.mkdir(parents=True, exist_ok=True)
    with out.open("w", encoding="utf-8", newline="") as f:
        w = csv.writer(f)
        headers = ["student"] + [f"q{i+1}" for i in range(max(len(qs) for _, qs in plan))]
        w.writerow(headers)
        for student, qs in plan:
            w.writerow([student, *qs])

# --- Run ----------------------------------------------------------------------
rng = random.Random(SEED)
extra_questions = _read_extra_questions(QUESTIONS_TXT)
students, name_col = _read_students(STUDENTS_CSV, NAME_FIELD)
plan = _make_plan(students, name_col, DEFAULT_QUESTIONS, extra_questions, EXTRA_RANDOM, rng)

if EXPORT_PATH:
    _export_plan(plan, EXPORT_PATH)
    print(f"Exported plan to: {EXPORT_PATH}")

print("\nWelcome round — press Enter for next student; type 'q' then Enter to stop.\n")
for i, (student, qs) in enumerate(plan, start=1):
    print(f"{i}/{len(plan)} — {student}")
    for j, q in enumerate(qs, start=1):
        print(f"  Q{j}: {q}")
    try:
        cmd = input("\n[Enter=next, q=quit] ")
    except EOFError:
        cmd = ""
    if cmd.strip().lower() == "q":
        print("Ending early. Have a great session!")
        break
    print()